### Passo 1 - Importar Arquivos e Bibliotecas

In [75]:
import pandas as pd
import os
import smtplib
import email.message
import ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

tabela_vendas = pd.read_excel('Bases de Dados/Vendas.xlsx')
tabela_emails = pd.read_excel('Bases de Dados/Emails.xlsx')
tabela_lojas = pd.read_csv('Bases de Dados/Lojas.csv', encoding='latin1', sep=';')

display(tabela_vendas)
display(tabela_emails)
display(tabela_lojas)

meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,17,Short Listrado,2,102,204
100995,69996,2019-12-26,17,Mochila,4,270,1080
100996,69996,2019-12-26,17,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,11,Camisa Listrado,1,108,108


,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,devbruno252+helena@gmail.com
1,Shopping Midway Mall,Alice,devbruno252+alice@gmail.com
2,Norte Shopping,Laura,devbruno252+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,devbruno252+manuela@gmail.com
4,Shopping União de Osasco,Valentina,devbruno252+valentina@gmail.com
5,Shopping Center Interlagos,Sophia,devbruno252+sophia@gmail.com
6,Rio Mar Recife,Isabella,devbruno252+isabella@gmail.com
7,Salvador Shopping,Heloisa,devbruno252+heloisa@gmail.com
8,Rio Mar Shopping Fortaleza,Luiza,devbruno252+luiza@gmail.com
9,Shopping Center Leste Aricanduva,Julia,devbruno252+julia@gmail.com


,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco
5,6,Shopping Center Interlagos
6,7,Rio Mar Recife
7,8,Salvador Shopping
8,9,Rio Mar Shopping Fortaleza
9,10,Shopping Center Leste Aricanduva


### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [76]:
tabela_vendas = tabela_vendas.merge(tabela_lojas, on= 'ID Loja')
data = tabela_vendas['Data'].max()

for id_loja, loja in tabela_lojas.values:
  os.makedirs(f'Backup Arquivos Lojas/{loja}', exist_ok= True)

  tabela = pd.DataFrame(tabela_vendas[tabela_vendas['ID Loja'] == id_loja])

  data_formatada = f'{data.day}_{data.month}_'
  
  caminho_save = f'Backup Arquivos Lojas/{loja}/{data_formatada}{loja}'
  tabela.to_excel(f'{caminho_save}.xlsx', index= False)

### Passo 3 - Salvar a planilha na pasta de backup

In [ ]:
def formatar_numero(numero):
    return f'R$ {numero:,.2f}'.replace(",", "X").replace(".", ",").replace("X", ".")

# msg = MIMEMultipart()
# msg["From"] = "@gmail.com"

context = ssl.create_default_context()

print(f"Conectando ao servidor SMTP...")
servidor = smtplib.SMTP("smtp.gmail.com", 587)
servidor.starttls(context=context)

print("TLS iniciado. Autenticando...")
servidor.login("@gmail.com", "")

print("Autenticação bem-sucedida. Enviando emails...")

for loja in os.listdir('Backup Arquivos Lojas'):
  for tabela in os.listdir('Backup Arquivos Lojas/' + loja):
    msg = MIMEMultipart()
    msg["From"] = "@gmail.com"
    tabela_temp = pd.read_excel(f'Backup Arquivos Lojas/{loja}/{tabela}')
    
    vendas_loja_dia = tabela_temp[tabela_temp['Data'] == data]
    
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    diversidade_dia = len(vendas_loja_dia['Produto'].unique())
    soma_vendas_dia = vendas_loja_dia[['Código Venda', 'Valor Final']].groupby('Código Venda').sum()
    ticket_medio_dia = soma_vendas_dia['Valor Final'].mean() if len(soma_vendas_dia) > 0 else 0

    faturamento_ano = tabela_temp['Valor Final'].sum()
    diversidade_ano = len(tabela_temp['Produto'].unique())
    soma_vendas_ano = tabela_temp[['Código Venda', 'Valor Final']].groupby('Código Venda').sum()
    ticket_medio_ano = soma_vendas_ano['Valor Final'].mean() if len(soma_vendas_ano) > 0 else 0

    dados_gerente = tabela_emails.loc[tabela_emails['Loja'] == loja].values[0]
    nome_gerente = dados_gerente[1]
    email_gerente = dados_gerente[2]

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if diversidade_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if diversidade_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    # ---------- email -----------

    msg["Subject"] = f'OnePage Dia {data.day}/{data.month} - Loja {loja}'
    msg["To"] = email_gerente

    corpo_email = f"""<p>Bom dia {nome_gerente},</p>
    <p>O resultado de ontem <strong>({data.day}/{data.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table style="width: 50%; border-collapse: collapse;">
    <tr>
        <th style="padding: 10px; text-align: left; border: none; background-color: #f2f2f2;"></th>
        <th style="padding: 10px; text-align: left; border: none; background-color: #f2f2f2;">Valor Dia</th>
        <th style="padding: 10px; text-align: left; border: none; background-color: #f2f2f2;">Meta Dia</th>
        <th style="padding: 10px; text-align: left; border: none; background-color: #f2f2f2;">Cenário Dia</th>
    </tr>
    <tr>
        <td style="padding: 10px; text-align: left; border: none;"><b>Indicador 1</b></td>
        <td style="padding: 10px; text-align: left; border: none;">{formatar_numero(faturamento_dia)}</td>
        <td style="padding: 10px; text-align: left; border: none;">{formatar_numero(meta_faturamento_dia)}</td>
        <td style="padding: 10px; text-align: left; border: none;"><font color="{cor_fat_dia}">◙</font></td>
    </tr>
    <tr>
        <td style="padding: 10px; text-align: left; border: none;"><b>Indicador 2</b></td>
        <td style="padding: 10px; text-align: left; border: none;">{diversidade_dia}</td>
        <td style="padding: 10px; text-align: left; border: none;">{meta_qtdeprodutos_dia}</td>
        <td style="padding: 10px; text-align: left; border: none;"><font color="{cor_qtde_dia}">◙</font></td>
    </tr>
    <tr>
        <td style="padding: 10px; text-align: left; border: none;"><b>Indicador 3</b></td>
        <td style="padding: 10px; text-align: left; border: none;">{formatar_numero(ticket_medio_dia)}</td>
        <td style="padding: 10px; text-align: left; border: none;">{formatar_numero(meta_ticketmedio_dia)}</td>
        <td style="padding: 10px; text-align: left; border: none;"><font color="{cor_ticket_dia}">◙</font></td>
    </tr>
    </table>

    <br>

    <table style="width: 50%; border-collapse: collapse;">
    <tr>
        <th style="padding: 10px; text-align: left; border: none; background-color: #f2f2f2;"></th>
        <th style="padding: 10px; text-align: left; border: none; background-color: #f2f2f2;">Valor Ano</th>
        <th style="padding: 10px; text-align: left; border: none; background-color: #f2f2f2;">Meta Ano</th>
        <th style="padding: 10px; text-align: left; border: none; background-color: #f2f2f2;">Cenário Ano</th>
    </tr>
    <tr>
        <td style="padding: 10px; text-align: left; border: none;"><b>Indicador 1</b></td>
        <td style="padding: 10px; text-align: left; border: none;">{formatar_numero(faturamento_ano)}</td>
        <td style="padding: 10px; text-align: left; border: none;">{formatar_numero(meta_faturamento_ano)}</td>
        <td style="padding: 10px; text-align: left; border: none;"><font color="{cor_fat_ano}">◙</font></td>
    </tr>
    <tr>
        <td style="padding: 10px; text-align: left; border: none;"><b>Indicador 2</b></td>
        <td style="padding: 10px; text-align: left; border: none;">{diversidade_ano}</td>
        <td style="padding: 10px; text-align: left; border: none;">{meta_qtdeprodutos_ano}</td>
        <td style="padding: 10px; text-align: left; border: none;"><font color="{cor_qtde_ano}">◙</font></td>
    </tr>
    <tr>
        <td style="padding: 10px; text-align: left; border: none;"><b>Indicador 3</b></td>
        <td style="padding: 10px; text-align: left; border: none;">{formatar_numero(ticket_medio_ano)}</td>
        <td style="padding: 10px; text-align: left; border: none;">{formatar_numero(meta_ticketmedio_ano)}</td>
        <td style="padding: 10px; text-align: left; border: none;"><font color="{cor_ticket_ano}">◙</font></td>
    </tr>
    </table>

    <p>Att., dev</p>
    """
    msg.attach(MIMEText(corpo_email, 'html'))

    with open(f'Backup Arquivos Lojas/{loja}/{tabela}', "rb") as arquivo:
        msg.attach(MIMEApplication(arquivo.read(), Name=tabela))

    servidor.send_message(msg)
  print(f'email enviado para {email_gerente}')    

print('Todos os emails enviados')
servidor.quit()

Conectando ao servidor SMTP...
TLS iniciado. Autenticando...
Autenticação bem-sucedida. Enviando emails...
email enviado para devbruno252+davi@gmail.com
email enviado para devbruno252+julia@gmail.com
email enviado para devbruno252+luiza@gmail.com
email enviado para devbruno252+sophia@gmail.com
email enviado para devbruno252+manuela@gmail.com
email enviado para devbruno252+livia@gmail.com
email enviado para devbruno252+lorenzo@gmail.com
email enviado para devbruno252+alice@gmail.com
email enviado para devbruno252+bernardo@gmail.com
email enviado para devbruno252+miguel@gmail.com
email enviado para devbruno252+marialuiza@gmail.com
email enviado para devbruno252+isabella@gmail.com
email enviado para devbruno252+cecilia@gmail.com
email enviado para devbruno252+lorena@gmail.com
email enviado para devbruno252+heitor@gmail.com
email enviado para devbruno252+theo@gmail.com
email enviado para devbruno252+helena@gmail.com
email enviado para devbruno252+valentina@gmail.com
email enviado para devb

(221,
 b'2.0.0 closing connection ada2fe7eead31-4e64e9c008asm10499405137.23 - gsmtp')

### Passo 4 - Calcular o indicador para 1 loja

In [59]:
ranking_ano = tabela_vendas[['Loja', 'Valor Final']].groupby('Loja').sum()
ranking_ano = ranking_ano.sort_values(by='Valor Final', ascending=False)

nome_arquivo = f'{data.day}_{data.month}_ranking_anual.xlsx'
ranking_ano.to_excel(f'Backup Arquivos Gerencia/{nome_arquivo}')
display(ranking_ano)

tabela_dia = tabela_vendas[tabela_vendas['Data'] == data]
ranking_dia = tabela_dia[['Loja', 'Valor Final']].groupby('Loja').sum()
ranking_dia = ranking_dia.sort_values(by= 'Valor Final', ascending=False)

nome_arquivo = f'{data.day}_{data.month}_ranking_diario.xlsx'
ranking_dia.to_excel(f'Backup Arquivos Gerencia/{nome_arquivo}')
display(ranking_dia)

,Valor Final
Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


,Valor Final
Loja,
Salvador Shopping,3950
Novo Shopping Ribeirão Preto,3400
Center Shopping Uberlândia,2651
Shopping Eldorado,2391
Shopping Center Interlagos,1582
Shopping Recife,1366
Norte Shopping,1259
Shopping União de Osasco,1207
Shopping Vila Velha,937


### Passo 5 - Enviar por e-mail para o gerente

In [ ]:
email_diretoria = tabela_emails.loc[tabela_emails['Loja'] == 'Diretoria', 'E-mail'].values[0]

msg = MIMEMultipart()
msg["From"] = "@gmail.com"

context = ssl.create_default_context()

print(f"Conectando ao servidor SMTP...")
servidor = smtplib.SMTP("smtp.gmail.com", 587)
servidor.starttls(context=context)

print("TLS iniciado. Autenticando...")
servidor.login(msg["From"], "")

print("Autenticação bem-sucedida. Enviando emails...")

msg["Subject"] = f'Ranking Dia {data.day}/{data.month}'
msg["To"] = email_diretoria

top_loja_dia = ranking_dia.index[0]
top_valor_dia = int(ranking_dia.iloc[0,0])
top_loja_ano = ranking_ano.index[0]
top_valor_ano = int(ranking_ano.iloc[0,0])

pior_loja_dia = ranking_dia.index[-1]
pior_valor_dia = int(ranking_dia.iloc[-1,0])
pior_loja_ano = ranking_ano.index[-1]
pior_valor_ano = int(ranking_ano.iloc[-1,0])

corpo_email = f'''
Prezados, bom dia

Melhor loja do Dia em Faturamento: Loja {top_loja_dia} com Faturamento {formatar_numero(top_valor_dia)}
Pior loja do Dia em Faturamento: Loja {pior_loja_dia} com Faturamento {formatar_numero(pior_valor_dia)}

Melhor loja do Ano em Faturamento: Loja {top_loja_ano} com Faturamento {formatar_numero(top_valor_ano)}
Pior loja do Ano em Faturamento: Loja {pior_loja_ano} com Faturamento {formatar_numero(pior_valor_ano)}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att.,
dev
'''

msg.attach(MIMEText(corpo_email))

with open(f'Backup Arquivos Gerencia/{data.day}_{data.month}_ranking_anual.xlsx', "rb") as arquivo1:
  msg.attach(MIMEApplication(arquivo1.read(), Name=f'{data.day}_{data.month}_ranking_anual.xlsx'))

with open(f'Backup Arquivos Gerencia/{data.day}_{data.month}_ranking_diario.xlsx', "rb") as arquivo2:
  msg.attach(MIMEApplication(arquivo2.read(), Name=f'{data.day}_{data.month}_ranking_diario.xlsx'))

servidor.send_message(msg)
print(f'email enviado para {email_diretoria}')    

servidor.quit()
    

Conectando ao servidor SMTP...
TLS iniciado. Autenticando...
Autenticação bem-sucedida. Enviando emails...
email enviado para devbruno252+diretoria@gmail.com


(221,
 b'2.0.0 closing connection 71dfb90a1353d-53074ad8c77sm11296360e0c.15 - gsmtp')

### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

### Passo 8 - Enviar e-mail para diretoria